In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SLDE /GEEAQCSABLKNTTOGAA)RLS ( NM', 'calculationPrice': 'close', 'open': 137.81, 'openTime': 1651563260532, 'openSource': 'fliifaco', 'close': 140.47, 'closeTime': 1620347438193, 'closeSource': 'oiiclffa', 'high': 139.68, 'highTime': 1619678180873, 'highSource': 'TILtX aadeE rs', 'low': 134.95, 'lowTime': 1687808407880, 'lowSource': 'e e5ltncddra iypie1 mue', 'latestPrice': 141.41, 'latestSource': 'Close', 'latestTime': 'February 12, 2021', 'latestUpdate': 1675694401715, 'latestVolume': 61503629, 'iexRealtimePrice': 137.9, 'iexRealtimeSize': 1, 'iexLastUpdated': 1619532858698, 'delayedPrice': 139.26, 'delayedPriceTime': 1645766868073, 'oddLotDelayedPrice': 141.7, 'oddLotDelayedPriceTime': 1632759657319, 'extendedPrice': 140.34, 'extendedChange': 0.06, 'extendedChangePercent': 0.00045, 'extendedPriceTime': 1628272322087, 'previousClose': 137.83, 'previousVolume': 65000772, 'change': 0.25, 'changePercent': 0.00187, 'volum

In [7]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

38.06

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data.status_code)
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
        )
        
#        print(symbol)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,129.44,58.06,N/A
1,AAL,17.57,-1.27,N/A
2,AAP,155.62,23.32,N/A
3,AAPL,138.02,37.69,N/A
4,ABBV,107.96,23.8,N/A
...,...,...,...,...
500,YUM,108.28,31.51,N/A
501,ZBH,165.76,-227.19,N/A
502,ZBRA,489.15,54.64,N/A
503,ZION,52.01,21.8,N/A


In [10]:
# Remove Glamour stocks, sort dataframe by peRatio in ascending order and get top 50

In [13]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
468,VLO,66.600,-2421.76,N/A
405,SBAC,271.080,-2120.96,N/A
380,PRGO,46.520,-937.1,N/A
183,FIS,141.200,-847.62,N/A
125,D,75.030,-493.52,N/A
...,...,...,...,...
363,PEG,61.110,None,N/A
442,TROW,166.720,None,N/A
452,UA,19.530,None,N/A
454,UAL,45.471,None,N/A


In [15]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,44.91,2.69,N/A
1,AIV,5.00,4.83,N/A
2,UNM,25.79,5.4,N/A
3,BIO,691.78,5.94,N/A
4,ALL,108.99,7.48,N/A
5,AFL,46.96,7.7,N/A
6,CPB,48.53,8.04,N/A
7,EBAY,64.75,8.22,N/A
8,KIM,18.31,8.97,N/A
9,MET,57.05,9.15,N/A


In [16]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [17]:
portfolio_input()

Enter the value of your portfolio:1000000


In [18]:
position_size = float(portfolio_size)/len(final_dataframe.index)
#position_size
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,44.91,2.69,445
1,AIV,5.00,4.83,4000
2,UNM,25.79,5.4,775
3,BIO,691.78,5.94,28
4,ALL,108.99,7.48,183
5,AFL,46.96,7.7,425
6,CPB,48.53,8.04,412
7,EBAY,64.75,8.22,308
8,KIM,18.31,8.97,1092
9,MET,57.05,9.15,350


In [19]:
# # Building the advanced model where many ratios would be considered
# Price-to-earnings ratio
# Price-to-book ratio
# Price-to-sales ratio
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
# Enterprise Value divided by Gross Profit (EV/GP)

In [20]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pr_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

{'beta': 1.1780022660922491, 'totalCash': 80545397199, 'currentDebt': 138760234614, 'revenue': 306809721547, 'grossProfit': 115139004626, 'totalRevenue': 306921845097, 'EBITDA': 88939413517, 'revenuePerShare': 17.76, 'revenuePerEmployee': None, 'debtToEquity': 5.518208165249224, 'profitMargin': 0.22388483197138573, 'enterpriseValue': 2319430134246, 'enterpriseValueToRevenue': 8.21, 'priceToSales': 7.89, 'priceToBook': 35.9, 'forwardPERatio': 31.137609588890104, 'pegRatio': 2.175121293169212, 'peHigh': 42.54602833201731, 'peLow': 17.2663164891823, 'week52highDate': '2021-01-17', 'week52lowDate': '2020-03-15', 'putCallRatio': 0.3356125321587141, 'companyName': 'Apple Inc', 'marketcap': 2328742275410, 'week52high': 146.83, 'week52low': 53.52, 'week52highSplitAdjustOnly': 146.78, 'week52highDateSplitAdjustOnly': '2021-01-10', 'week52lowSplitAdjustOnly': 55.32, 'week52lowDateSplitAdjustOnly': '2020-03-22', 'week52change': 0.696793070164884, 'sharesOutstanding': 17348833715, 'float': None, '

In [21]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A',   
            
        ],
        index = rv_columns),
        ignore_index = True
        )

In [22]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.44,N/A,56.81,N/A,8.13,N/A,7.39,N/A,35.454907,N/A,13.961229,N/A,N/A
1,AAL,17.70,N/A,-1.23,N/A,-1.96,N/A,0.4351,N/A,-6.081270,N/A,1.400774,N/A,N/A
2,AAP,159.97,N/A,22.59,N/A,2.81,N/A,1.07,N/A,10.698247,N/A,2.406812,N/A,N/A
3,AAPL,140.58,N/A,37.61,N/A,35.41,N/A,7.87,N/A,27.876426,N/A,20.878004,N/A,N/A
4,ABBV,106.38,N/A,23.3,N/A,12.29,N/A,4.56,N/A,15.474702,N/A,9.350017,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.41,N/A,31.69,N/A,-4.17,N/A,5.84,N/A,23.190993,N/A,10.240829,N/A,N/A
501,ZBH,161.55,N/A,-220.46,N/A,2.83,N/A,4.73,N/A,27.885377,N/A,8.411886,N/A,N/A
502,ZBRA,493.49,N/A,56.02,N/A,13.46,N/A,5.9,N/A,37.441127,N/A,14.137954,N/A,N/A
503,ZION,51.98,N/A,21.3,N/A,1.19,N/A,2.78,N/A,12.231040,N/A,3.198604,N/A,N/A
